In [ ]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()

Program bitfile using platform cable or other means!

Note for this to work, CW305.py requires an edit to block programming, in `_con()`, under the `if 'ss2' in self.platform:` block: comment out the `self.fpga.program` calls:
```python
            #if self.platform == 'ss2_ice40':
            #    self.fpga.erase_and_init()
            #    self.fpga.program(bsfile, sck_speed=prog_speed, start=True, use_fast_usb=False)
            #else:
            #    self.fpga.program(bsfile, sck_speed=prog_speed)
```

and also this:
```python
#self.fpga = CW312T_XC7A35T(scope)
```

In [ ]:
# Connect to target (but don't program it):
# warning about right number of defines is safe to ignore:
target = cw.target(scope, cw.targets.CW305, force=False, bsfile='', platform='ss2_a35', defines_files=['../fpga/hdl/sonata_defines.v'])

Diagnostic LEDs: by default, LEDs show the following:
- `USRLED0`: main clock heartbeat (25 MHz, MAINCLK on schematic)
- `USRLED1`: programmable clock heartbeat (MAINCLK multiplied by 4 (default) by FPGA PLL)
- `USRLED2`: settable via register write to `target.REG_USER_LED`
- `USRLED3`: hyperram controller stuck
- `USRLED4`: hyperram controller 90 degree shifted clock is unlocked
- `USRLED5`: reset
- `USRLED6`: programmable clock MMCM is locked
- `USRLED7`: ss2 error
- `CHERIERR0`: XADC error (temp/voltage out of spec)
- all other LEDs off

Next we check that we can communicate with the Sonata FPGA.

**Uses SimpleSerial2 using scope's tio1/tio2 and Sonata's SER0_TX/SER0_RX (on P12 pins 2 and 4).**

**If this fails, nothing else will work!**

In [ ]:
target._ss2_test_echo()

In [ ]:
# you should also see a build time that makes sense:
target.fpga_buildtime

# LEDs and switches:

In [ ]:
# make `USERLED2` blink:
import time
for i in range(10):
    target.fpga_write(target.REG_USER_LED, [1])
    time.sleep(0.2)
    target.fpga_write(target.REG_USER_LED, [0])
    time.sleep(0.2)

In [ ]:
# make all the LEDs blink:
target.fpga_write(target.REG_LEDS, [0, 0, 0x60])

In [ ]:
# return LEDs to normal:
target.fpga_write(target.REG_LEDS, [0, 0, 0])

In [ ]:
# read status of input switches:
raw = int.from_bytes(target.fpga_read(target.REG_DIPS, 2), byteorder='little')
for i in range(0,16):
    if i in range(0,3): label = 'SELSW%d' % i
    elif i in range(3, 8): label = 'NAVSW%d' % (i-3)
    elif i in range(8, 16): label = 'USRSW%d' % (i-8)
    print(label, end=': ')
    if raw & 2**i:
        print('on')
    else:
        print('off')
    
    if i in [2, 7]:
        print()

This runs in an infinite loop, updating what's shown whenever a switch status changes:

In [ ]:
from IPython.display import display, clear_output

prev_raw = 0
while True:
    raw = int.from_bytes(target.fpga_read(target.REG_DIPS, 2), byteorder='little')
    if raw != prev_raw:
        prev_raw = raw
        clear_output()
        for i in range(0,16):
            if i in range(0,3): label = 'SELSW%d: ' % i
            elif i in range(3, 8): label = 'NAVSW%d: ' % (i-3)
            elif i in range(8, 16): label = 'USRSW%d: ' % (i-8)
        
            if raw & 2**i:
                label += 'on'
            else:
                label += 'off'
            display(label)

            if i in [2, 7]:
                display('')


# XADC:

In [ ]:
# connect XADC:
from chipwhisperer.capture.scopes.cwhardware.ChipWhispererHuskyMisc import XADCSettings
from chipwhisperer.common.utils import util

CODE_READ       = 0x80
CODE_WRITE      = 0xC0

class fakeoa(object):
    def __init__(self):
        pass    
    def sendMessage(self, mode, address, payload=None, Validate=False, maxResp=1, readMask=None):
        if mode != CODE_WRITE:
            return target.fpga_read(address, maxResp)
        else:
            target.fpga_write(address, payload)

oa = fakeoa()
xadc = XADCSettings(oa)
xadc.drp.data = target.REG_XADC_DRP_DATA
xadc.drp.addr = target.REG_XADC_DRP_ADDR
xadc.drp.reset_address = None

def xadc_status():
    self = xadc
    rtn = {}
    rtn['current temperature [C]'] = '%.1f' % self.temp
    rtn['maximum temperature [C]'] = '%.1f' % self.max_temp
    rtn['user temperature alarm trigger [C]'] = '%.1f' % self.temp_trigger
    rtn['user temperature reset trigger [C]'] = '%.1f' % self.temp_reset
    rtn['device temperature alarm trigger [C]'] = '%.1f' % self.ot_temp_trigger
    rtn['device temperature reset trigger [C]'] = '%.1f' % self.ot_temp_reset
    rtn['vccint'] = '%.3f' % self.vccint
    rtn['vccaux'] = '%.3f' % self.vccaux
    rtn['vccbram'] = '%.3f' % self.vccbram
    
    rtn['max vccint'] = '%.3f' % self.get_vcc('vccint', 'max')
    rtn['max vccaux'] = '%.3f' % self.get_vcc('vccaux', 'max')
    rtn['max vccbram'] = '%.3f' % self.get_vcc('vccbram', 'max')

    rtn['min vccint'] = '%.3f' % self.get_vcc('vccint', 'min')
    rtn['min vccaux'] = '%.3f' % self.get_vcc('vccaux', 'min')
    rtn['min vccbram'] = '%.3f' % self.get_vcc('vccbram', 'min')

    print(util.dict_to_str(rtn))

def xadc_clear_error():
    target.fpga_write(target.REG_XADC_STAT, [0])

We can't print the `xadc` object because it's a Husky class with more things that we have here; individual temperature and voltage measurements can be accessed as usual, or used `xadc_status()` to print everything:

In [ ]:
xadc.temp, xadc.max_temp

In [ ]:
xadc.vccint, xadc.vccaux, xadc.vccbram

In [ ]:
xadc_status()

If the temperature specified by the "user temperature alarm trigger" is exceeded, the programmable clock (which runs the AES engines and the hyperram controller) gets shut down in an effort to preserve the FPGA.

This threshold can be controlled via the `xadc.temp_trigger` property.

Regardless of this user-defined threshold, the "device temperature alarm trigger", which is an immutable build-time property (nominally 85C), also shuts down the clock.

The clock is also shutdown if any of the VCC supplies exceed their recommended operating range (you can use the `xadc._get_vcc_limit()` and `xadc._set_vcc_limit()` methods to alter these).

The clock used for the serial communication link is always kept alive, so it should (in theory!) be possible to recover from a clock shutdown without a reset.

XADC errors are sticky; run `xadc_clear_error()` to un-stick them, which should re-enable the clock (if the error condition is no longer present).

In [ ]:
xadc_clear_error()

## Reading ANALOGx (VAUX) signals:

In [ ]:
def xadc_analog(i=0):
    if   i == 0: j = 4
    elif i == 1: j = 12
    elif i == 2: j = 5
    elif i == 3: j = 13
    elif i == 4: j = 6
    elif i == 5: j = 14
    else: raise ValueError('Unsupported index')
    addr = 0x10 + j
    raw = xadc.drp.read(addr)
    return (raw>>4)/4096 # ref: UG480

In [ ]:
for i in range(6):
    print('ANALOG%d: %f' % (i, xadc_analog(i)))

## Reading the ANALOGx_DIGITAL signals:

In [ ]:
raw = target.fpga_read(target.REG_ANALOG_DIGITAL, 1)[0]
for i in range(0,6):
    print('ANALOG%d_DIGITAL: ' % i, end='')
    if raw & 2**i:
        print('high')
    else:
        print('low')

# Setting the programmable clock:

This is the clock used by the AES cores and the Hyperram controller. Default setting is 100 MHz.

In [ ]:
# Connect MMCM:
from chipwhisperer.capture.scopes.cwhardware.ChipWhispererHuskyMisc import XilinxDRP, XilinxMMCMDRP
drp = XilinxDRP(oa, target.REG_MMCM_DRP_DATA, target.REG_MMCM_DRP_ADDR, target.REG_MMCM_DRP_RESET)
mmcm = XilinxMMCMDRP(drp)

The programmable clock can be controlled via its MMCM's divider and multiplier settings.

The input clock is 25 MHz and legal VCO range is around 600 - 1200 MHz IIRC:

In [ ]:
mmcm.get_main_div()

In [ ]:
mmcm.get_mul()

In [ ]:
mmcm.get_sec_div()

For example, this sets the clock to 25 * 50 / 11 = 113.6 MHz:

In [ ]:
mmcm.set_mul(50)
mmcm.set_sec_div(11)

**Be careful with these settings as it is possible to set the clock so high that it could overheat and even damage the FPGA!**

# Running AES:

In [ ]:
# run AES:
# there are twelve AES cores; each bit turns on/off the corresponding core:
CORES = 0b111111111111
target.fpga_write(target.REG_AES_CORES_EN, int.to_bytes(CORES, length=8, byteorder='little'))

# write some key and PT:
target.fpga_write(target.REG_CRYPT_KEY, range(16))
target.fpga_write(target.REG_CRYPT_TEXTIN, range(16))
target.fpga_write(target.REG_CRYPT_GO, [1])

# read result:
print(list(target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)))

To stress the board power supply and heat dissipation, run this to make it encrypt non-stop. **Keep an eye out on the XADC temperature!**

In [ ]:
# make it encrypt non-stop:
target.fpga_write(target.REG_AES_ALWAYS_ON, [1])

In [ ]:
# and stop:
target.fpga_write(target.REG_AES_ALWAYS_ON, [0])

# Hyperram test:

Note this is the same IP and interface used on the Luna board.

In [ ]:
from tqdm.notebook import tqdm
import time

def lb_write(addr, data1, data2=None):
    # 1. set lb_action(s) to write:
    action_bits = 0
    if data1 != None:
        action_bits += 2**0
    if data2 != None:
        action_bits += 2**2
    target.fpga_write(target.REG_LB_ACTION, [action_bits])
    
    # 2. write data(s)
    if data1 != None and data2 != None:
        target.fpga_write(target.REG_LB_DATA1, int.to_bytes(data1 + (data2 << 32), length=8, byteorder='little'))
    elif data1 != None:
        target.fpga_write(target.REG_LB_DATA1, int.to_bytes(data1, length=4, byteorder='little'))
    elif data2 != None:
        target.fpga_write(target.REG_LB_DATA2, int.to_bytes(data2, length=4, byteorder='little'))
    else:
        raise ValueError('Must specify some data to write!')
    
    # 3. write address (this triggers the LB write action):
    target.fpga_write(target.REG_LB_ADDR, int.to_bytes(addr, length=4, byteorder='little'))
        

def lb_read(addr, h1en=True, h2en=False):
    # 1. set lb_action(s) to read:
    action_bits = 0
    if h1en:
        action_bits += 2**1
    if h2en:
        action_bits += 2**3
    target.fpga_write(target.REG_LB_ACTION, [action_bits])

    # 2. write address (this triggers the LB read action):
    target.fpga_write(target.REG_LB_ADDR, int.to_bytes(addr, length=4, byteorder='little'))
    
    # 3. read data(s):
    data1 = None
    data2 = None
    if h1en and h1en:
        raw = target.fpga_read(target.REG_LB_DATA1, 8)
        data1 = int.from_bytes(raw[0:4], byteorder='little')
        data2 = int.from_bytes(raw[4:8], byteorder='little')
    elif h1en:
        data1 = int.from_bytes(target.fpga_read(target.REG_LB_DATA1, 4), byteorder='little')
    elif h2en:
        data2 = int.from_bytes(target.fpga_read(target.REG_LB_DATA2, 4), byteorder='little')
    else:
        raise ValueError('Must read from at least one memory!')

    return data1, data2

def set_config(config=0x8fec0000):
    # set config as per do_files:
    lb_write(0x14, config, config) # default = 83 MHz fixed 2x access
    lb_write(0x1c, 0x05, 0x05) # execute the config write

def reset_hyperram(short_pulse=False):
    target.fpga_write(target.REG_LB_MANUAL, [1]) # turn off auto test (in case FSM was stuck)
    if short_pulse:
        target.fpga_write(target.REG_HYPER_RESET, [0xff])
    else:
        target.fpga_write(target.REG_HYPER_RESET, [1])
        target.fpga_write(target.REG_HYPER_RESET, [0])

def write_mem_1word(addr, data1, data2=None, h1en=True, h2en=False):
    if h1en:
        addr1=addr
        cmd1 = 0x01
    else:
        addr1=None
        cmd1 = None
    if h2en:
        addr2=addr
        cmd2 = 0x01
    else:
        addr2=None
        cmd2 = None
    
    lb_write(0x10, addr1, addr2) # address
    lb_write(0x14, data1, data2) # data
    lb_write(0x1c, cmd1, cmd2) # write command

def read_mem_1word(addr, h1en=True, h2en=False):
    if h1en:
        addr1=addr
        cmd1=0x04
    else:
        addr1=None
        cmd1=None
    if h2en:
        addr2=addr
        cmd2=0x04
    else:
        addr2=None
        cmd4=None
    
    lb_write(0x10, addr1, addr2) # address
    lb_write(0x1c, cmd1, cmd2) # read command
    return lb_read(0x14, h1en, h2en)

def write_mem_words(addr, data):
    # write 64 bits to each hyperram (total 128 bits)
    # data should be a list of four 32-bit ints
    cmd = 0x3 
    lb_write(0x10, addr, addr) # address
    lb_write(0x14, data[0], data[1])
    lb_write(0x18, data[2], data[3])
    lb_write(0x1c, cmd, cmd) # write command

def read_mem_words(addr, h1en=True, h2en=False):
    # read 64 bits from each hyperram (total 128 bits)
    # returns a list of four 32-bit ints
    cmd = 0x04  
    lb_write(0x10, addr, addr) # address
    lb_write(0x1c, cmd, cmd) # read command
    data = list(lb_read(0x14, True, True))
    data.extend(list(lb_read(0x18, True, True)))
    return data

def auto_hyperram_test(hr1en=True, hr2en=False, start=0, stop=64*1024*1024//32-1):
    target.fpga_write(target.REG_LB_START_ADDR, int.to_bytes(start, length=4, byteorder='little'))
    target.fpga_write(target.REG_LB_STOP_ADDR, int.to_bytes(stop,  length=4, byteorder='little'))
    target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test and clear fail
    config = 16 # set LFSR mode
    if hr1en:
        config += 8
    if hr2en:
        config += 4
    target.fpga_write(target.REG_LB_MANUAL, [config]) # turn on test
    wpbar = tqdm(total=stop, desc='writes')
    prevaddr = 0
    addr = 0
    while addr < stop:
        addr = int.from_bytes(target.fpga_read(target.REG_LB_CURRENT_ADDR, 4), byteorder='little') # current address being read/written
        if addr < prevaddr:
            break
        wpbar.update(addr - prevaddr)
        prevaddr = addr
        time.sleep(0.1)
        busy_stuck = target.fpga_read(target.REG_HYPER_STATUS, 1)[0] & 6
        if busy_stuck:
            raise ValueError("Hyperram controller appears to be stuck! Re-run the reset_hyperram() and set_config() cells to reset it before trying again.")
    wpbar.close()
    
    rpbar = tqdm(total=stop, desc='reads')
    prevaddr = 0
    addr = 0
    while addr < stop:
        addr = int.from_bytes(target.fpga_read(target.REG_LB_CURRENT_ADDR, 4), byteorder='little') # current address being read/written
        if addr < prevaddr:
            break
        rpbar.update(addr - prevaddr)
        prevaddr = addr
        time.sleep(0.1)
        busy_stuck = target.fpga_read(target.REG_HYPER_STATUS, 1)[0] & 6
        if busy_stuck:
            raise ValueError("Hyperram controller appears to be stuck! Re-run the reset_hyperram() and set_config() cells to reset it before trying again.")

    total_errors = int.from_bytes(target.fpga_read(target.REG_LB_ERRORS, 4), byteorder='little')
    rpbar.close()
    
    status = target.fpga_read(target.REG_HYPER_STATUS, 1)[0]

    if status & 16:
        print('Test passed')
    else:
        print('Test FAILED; %d errors' % total_errors)
        if hr1en and hr2en:
            mx = 2
        else:
            mx = 1
        print('Percentage of good reads: %3.2f%%' % ((1-(total_errors/((stop-start)*mx)))*100))
    

def auto_last_address():
    return int.from_bytes(target.fpga_read(target.REG_LB_CURRENT_ADDR, 4), byteorder='little')

def auto_test_errors():
    return int.from_bytes(target.fpga_read(target.REG_LB_ERRORS, 4), byteorder='little')

def auto_hyperram_long(start=0, stop=64*1024*1024//32-1, iterations = 1000):
    target.fpga_write(target.REG_LB_START_ADDR, int.to_bytes(start, length=4, byteorder='little'))
    target.fpga_write(target.REG_LB_STOP_ADDR, int.to_bytes(stop,  length=4, byteorder='little'))
    target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test and clear fail
    config = 16 + 8 # set LFSR mode, enable HR1 only
    target.fpga_write(target.REG_LB_MANUAL, [config]) # turn on test
    pbar = tqdm(total=iterations, desc='iterations')
    ebar = tqdm(total=1000, desc='errors')
    prev_iteration = 0
    current_iteration = 0

    prev_errors = 0
    current_errors = 0

    while current_iteration < iterations:
        current_iteration = int.from_bytes(target.fpga_read(target.REG_LB_ITERATIONS, 2), byteorder='little')
        pbar.update(current_iteration - prev_iteration)
        prev_iteration = current_iteration

        current_errors = int.from_bytes(target.fpga_read(target.REG_LB_ERRORS, 4), byteorder='little')
        ebar.update(current_errors - prev_errors)
        prev_errors = current_errors

        time.sleep(0.01)
        busy_stuck = target.fpga_read(target.REG_HYPER_STATUS, 1)[0] & 6
        if busy_stuck:
            raise ValueError("Hyperram controller appears to be stuck! Re-run the reset_hyperram() and set_config() cells to reset it before trying again.")
    pbar.close()
    ebar.close()

    status = target.fpga_read(target.REG_HYPER_STATUS, 1)[0]

    if status & 16:
        print('Test passed')
    else:
        print('Test FAILED; %d errors' % current_errors)
        print('Percentage of good reads: %3.2f%%' % ((1-(current_errors/((stop-start)*iterations)))*100))


Whenever the clock rate is changed, you should re-run the `reset_hyperram()` and `set_config()` commands:

In [ ]:
reset_hyperram()

In [ ]:
#set_config(0x8f140a07) # 166 MHz variable
#set_config(0x8f1c0a07) # 166 MHz fixed
#set_config(0x8f0c0806) # 133 MHz fixed

set_config(0x8ffc0605) # 100 MHz fixed
#set_config(0x8fec0404) # 83 MHz fixed

Automated check over the full memory space (single iteration):

In [ ]:
auto_hyperram_test()

In [ ]:
# If stuck, report last read or write address, and number of errors:
print('Last read or write address (if stuck): %d' % auto_last_address())
print('Numer of errors: %d' % auto_test_errors())

In [ ]:
# can also limit the address range; here does first 8192 words only:
auto_hyperram_test(stop=8192//32-1)

Stress test by running multiple iterations over the full memory space (takes roughly 1 second per iteration):

In [ ]:
auto_hyperram_long(iterations=100)

Python-driven memory test -- much slower; can be useful to diagnose issues.

Unlike the automated test this does not validate the full memory (it would take a few days with this approach!), however it does fully exercise the interface of the memory (with the default settings used below).

In [ ]:
WORDS = 32
#WORDS = 1024
# CHECKS controls which of the two words per burst are verified:
CHECKS = [0,2] # check burst read/write
#CHECKS = [0] # check only the first 32 bits

# start address for the test (divide what auto_last_address() reports by 2):
START = 0
#START = 233000//2

In [ ]:
import random
from tqdm.notebook import tnrange

target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test and clear fail

wdata = []
errors = 0
goods = 0
for i in tnrange(START, START+WORDS, desc='Writing'):
    data = []
    for j in range(4):
        data.append(random.randint(0, 2**32-1))
        #print('i=%3d, j=%3d: writing %08x' % (i, j, data[-1]))
    write_mem_words(i*2, data)
    wdata.append(data)
    busy_stuck = target.fpga_read(0x83, 1)[0] & 6
    if busy_stuck:
        raise ValueError("Hyperram controller appears to be stuck on address %08x! Re-run the reset_hyperram() and set_config() cells to reset it before trying again." % (i*2))


for i in tnrange(START, START+WORDS, desc='Reading'):
    rdata = read_mem_words(i*2)
    busy_stuck = target.fpga_read(0x83, 1)[0] & 6
    if busy_stuck:
        raise ValueError("Hyperram controller appears to be stuck on address %08x! Re-run the reset_hyperram() and set_config() cells to reset it before trying again." % (i*2))
    rdata_filtered = []
    wdata_filtered = []
    for j in CHECKS:
        rdata_filtered.append(rdata[j])
        wdata_filtered.append(wdata[i-START][j])                     
    if rdata_filtered != wdata_filtered:
        for j in range(len(CHECKS)):
            expect = wdata_filtered[j]
            got = rdata_filtered[j]
            if got != expect:
                errors += 1
                if errors < 10:
                    print('i=%3d, j=%3d: got %08x, expected %08x, diff %08x, bits wrong: %3d' % (i, j, got, expect, got ^ expect, bin(got ^ expect).count('1')))
            else:
                goods += 1
    else:
        if goods == 0:
            print('Good read for i=%3d' % i)
        goods += len(CHECKS)

print('Percentage of good reads: %d%%' % (goods/(errors+goods)*100))
assert errors == 0, '%d errors!' % errors